In [1]:
%%datalabframework getfilename

In [2]:
import datalabframework as dlf
logger = dlf.log.initLogger(__name__, kafka_topic="datalab", kafka_servers="kafka:9092")

2017-04-30 14:24:41,070 - jovyan - __main__ - INFO - init - {'notebook': {'filepath': '/home/jovyan/work/notebooks/validate', 'filename': 'validate.ipynb'}, 'project': {'rootpath': '/home/jovyan/work/notebooks', 'main': 'main.ipynb'}, 'datalab': {'framework': '0.1'}}


In [3]:
# EXPORT

from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [4]:
# EXPORT

def auc(df, model):
    prediction = model.transform(df)
    evaluator = BinaryClassificationEvaluator()

    metric = evaluator.evaluate(prediction)
    metric_name = evaluator.getMetricName()

    return metric_name, metric

In [5]:
DATA_ROOT = '/home/jovyan/work/data'
d = {   
    'input_sample' : 1.0,
    'input_source' : DATA_ROOT + '/titanic/data/set/train/features.parquet',
    'input_model' : DATA_ROOT + '/titanic/models/lr.model'
}

p = dlf.params.config_fromdict(d)

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(dlf.project.filename()).getOrCreate()
spark.version

'2.1.0'

In [9]:
df = spark.read.parquet(p.input_source)
df.printSchema()
df.show(10, truncate=False)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Title: string (nullable = true)

+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|PassengerId|Survived|Pclass|Sex   |Age               |SibSp|Parch|Fare   |Embarked|Title |
+-----------+--------+------+------+------------------+-----+-----+-------+--------+------+
|1          |0       |3     |male  |22.0              |1    |0    |7.25   |S       |Mr    |
|2          |1       |1     |female|38.0              |1    |0    |71.2833|C       |Mrs   |
|3          |1       |3     |female|26.0              |0    |0    |7.925  |S       |Miss  |
|4          |1       |1     |female|35.0              |1   

In [10]:
from features import features
df_features = features.feature_vector(df, 
                  'PassengerId', 
                  'Survived', 
                  ['Pclass', 
                   'Sex', 
                   'Age', 
                   'SibSp', 
                   'Parch',
                   'Fare', 
                   'Embarked', 
                   'Title'])

importing Jupyter notebook from /home/jovyan/work/notebooks/features/features.ipynb


In [11]:
from pyspark.ml.classification import LogisticRegressionModel

model = LogisticRegressionModel.load(p.input_model)

In [12]:
metric_name, metric = auc(df_features, model)


In [13]:
print("Evaluation {} : {}".format(metric_name, metric))

Evaluation areaUnderROC : 0.8742077567933189
